In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from tqdm import tqdm

# Even-odd Hamiltonian - $L=4$

In [2]:
Z = np.array([[1,0],[0,-1]])
Y = np.array([[0,-1.j],[1.j,0]])
X = np.array([[0,1],[1,0]])
I = np.eye(2)

def kron(ops):
    op = 1
    for o in ops:
        op = np.kron(op, o)
    return op

In [3]:
# TFI Hamiltonian
Jz = 1
gx = 1
gz = 0
h = Jz*np.kron(Z, Z) + gx/2 * (np.kron(X, I) + np.kron(I, X)) + gz/2 * (np.kron(Z, I) + np.kron(I, Z)) 

In [4]:
h

array([[ 1. ,  0.5,  0.5,  0. ],
       [ 0.5, -1. ,  0. ,  0.5],
       [ 0.5,  0. , -1. ,  0.5],
       [ 0. ,  0.5,  0.5,  1. ]])

In [5]:
L, U = np.linalg.eigh(h)
GS = U[:,0]
print(L)
print(GS.conj().T @ h @ GS)

[-1.41421356 -1.          1.          1.41421356]
-1.4142135623730936


In [6]:
psi0 = np.kron(GS, GS)

In [7]:
psi0.conj().T @ (np.kron(h, np.eye(4)) + np.kron(np.eye(4), h)) @ psi0

-2.8284271247461845

In [8]:
L[0]*2

-2.8284271247461885

In [9]:
psi0.reshape(2,2,2,2)

array([[[[ 0.0732233, -0.1767767],
         [-0.1767767,  0.0732233]],

        [[-0.1767767,  0.4267767],
         [ 0.4267767, -0.1767767]]],


       [[[-0.1767767,  0.4267767],
         [ 0.4267767, -0.1767767]],

        [[ 0.0732233, -0.1767767],
         [-0.1767767,  0.0732233]]]])

1. Construct 3 qubit block encoding
2. Apply this to enlarged state of 1.5 * L qubits
3. Determine measurement probability from norm of states
4. 

In [18]:
# Function that does many steps
def HITE(psi, beta, h, max_iter, adaptive=True, force_fail=False, verbose=False, seed=0, periodic=True, correct_iter=3, kill=0):
    # psi is L leg tensor
    # h is two qubit translationally invariant Hamiltonian term
    if seed >= 0:
        np.random.seed(seed)
    d = h.shape[0]
    ell = psi.ndim
    
    L, U = np.linalg.eigh(h) # Energy eigenbasis
    GS = U[:,0]
    Ut = U.reshape(2,2,2,2) # o1, o2, i1, i2
    if ell % 2 == 0:
        # Even number of sites; make pair GS
        GSP = kron([GS] * (ell//2))

    """
    Let a be the minimum of the spectrum. We want 1 - exp(-beta (a + shift)) = kill.
    So when kill = 0, we want shift = -a; then when we fail a measurement, we kill the ground state.
    In general, shift = log(1 - kill) / (-beta) - a
    """
    # Shift spectrum of H to be non-negative with GS energy being 0
    assert kill == 0
    h_diag = np.diag(L)
    shift = np.log(1 - kill) / (-beta) - np.min(L)
    h_diag += shift * np.eye(d)
    #h_diag -= np.min(L) * np.eye(d)
    if verbose:
        print(f"Shifted Spectrum with shift {shift}: {np.diag(h_diag)}.")
    assert np.isclose((U.conj().T @ GS).conj().T @ h_diag @ (U.conj().T @ GS), np.min(L) + shift), GS.conj().T @ h_diag @ GS
    
    # Make gamma = e^{-beta H_diag)
    gamma = sp.linalg.expm(-beta * h_diag)
    gammat = gamma.reshape(2,2,2,2)
    gamma_bar = np.sqrt(np.eye(d) - gamma**2)
    gamma_bart = gamma_bar.reshape(2,2,2,2)
    
    U_gamma = np.zeros((2*d,2*d), dtype=gamma.dtype)
    U_gamma[:d,:d] = gamma
    U_gamma[d:,:d] = U_gamma[:d,d:] = gamma_bar
    U_gamma[d:,d:] = -gamma
    assert np.isclose(np.linalg.norm(U_gamma @ U_gamma.conj().T - np.eye(2*d)), 0.0)

    assert periodic==True
    h_terms = [h] + [I]*(ell-2)
    h_big = kron(h_terms)
    h_big = h_big.reshape([2]*(2*ell))
    H = 0
    for i in range(ell - (not periodic)):
        H += h_big
        h_big = h_big.transpose(list(range(1, ell)) + [0] + list(range(ell+1, 2*ell)) + [ell])
        
    H = H.reshape(2**ell, -1)

    LW, UW = np.linalg.eigh(H) # Energy eigenbasis
    if verbose:
        print(f"GS Energy: {LW[0]}.")
    GSW = UW[:,0]

    psiF = psi.flatten()
    Es = [[[psiF.conj().T @ H @ psiF]]] # Initial energy
    Fs = [[[np.abs(GSW.conj().T @ psiF)**2]]]
    Fs_P = [[[np.abs(GSP.conj().T @ psiF)**2]]]
    
    if verbose:
        print(f"Energy at iteration {0}: {Es[-1]}.")
        print(f"Fidelity at iteration {0}: {Fs[-1]}.")
        print(f"Pair fidelity at iteration {0}: {Fs_P[-1]}.")
        
    mos = []
    def permutation(fail_count, d):
        # Define one particular choice of permutation matrix
        P = np.eye(d)
        if fail_count == 0:
            P = np.roll(np.eye(d), 1, axis=1)
        else:
            P[:-fail_count,:-fail_count] = np.roll(P[:-fail_count,:-fail_count], 1, axis=1)
        return P
    for j in (tqdm(range(1, max_iter+1)) if verbose else range(1, max_iter+1)):
        E_layer = []
        F_layer = []
        FP_layer = []
        mo_layer = []
        for i in range(0, ell, 2):
            E_gate = []
            F_gate = []
            FP_gate = []
            mo_gate = []
            fail_count = 0
            #for k in range(4):
            k = 0
            ci_local = correct_iter
            while k < ci_local:
                # apply gate to (i, i+1)
    
                # First move to local 2-site energy eigenbasis.
                psi = np.tensordot(Ut.conj(), psi, axes=([0,1],[i,i+1])) # i1, i2, 0, ..., i-1, i+2, ..., L-1
                reorder = list(range(2, i+2)) + [0,1] + list(range(i+2, ell))
                psi = psi.transpose(reorder)
    
                gamma_psi = np.tensordot(gammat, psi, axes=([2,3],[i,i+1])) # i1, i2, 0, ..., i-1, i+2, ..., L-1
                gamma_psi = gamma_psi.transpose(reorder)
                gpF = gamma_psi.flatten()
                p0 = np.linalg.norm(gpF)**2
                p1 = 1 - p0
                if p1 < 1.e-14:
                    p1 = 0
                    p0 = 1
                mo = np.random.choice([0,1], size=1, p=[p0,p1])[0]
                if mo == 1:
                    # Want to see 3 successful measurements before going on.
                    k = 0
                    ci_local *= 2
                if p1 > 1.e-10 and force_fail:
                    mo = 1
                mo_gate.append(mo)
                if verbose:
                    print(f"Measurement outcome and probability at round {k}: {mo}, {[p0,p1][mo]}.")
    
                if mo == 0:
                    psi = gamma_psi
                else:
                    gamma_psi = np.tensordot(gamma_bart, psi, axes=([2,3],[i,i+1])) # i1, i2, 0, ..., i-1, i+2, ..., L-1
                    gamma_psi = gamma_psi.transpose(reorder)
                    psi = gamma_psi
                assert np.isclose(np.linalg.norm(psi)**2, [p0,p1][mo])
                psi /= np.linalg.norm(psi)
    
                if mo == 1 and adaptive:
                    # Made measurement error; try to fix
                    nP = permutation(fail_count, d)
                    assert np.isclose(np.linalg.norm(nP @ nP.conj().T - np.eye(d)), 0.0)
                    nP = nP.reshape(2,2,2,2)
                    psi = np.tensordot(nP, psi, axes=([2,3],[i,i+1])) # i1, i2, 0, ..., i-1, i+2, ..., L-1
                    psi = psi.transpose(reorder)
                    fail_count += 1
                
                # Move back to compoutational basis on 2-qubit subspace
                psi = np.tensordot(Ut, psi, axes=([2,3],[i,i+1])) # i1, i2, 0, ..., i-1, i+2, ..., L-1
                psi = psi.transpose(reorder)
    
                psiF = psi.flatten()
                E_gate.append(psiF.conj().T @ H @ psiF)
                F_gate.append(np.abs(GSW.conj().T @ psiF)**2)
                FP_gate.append(np.abs(GSP.conj().T @ psiF)**2)
                
                if len(E_gate) > 1 and np.abs(E_gate[-1] - E_gate[-2]) < 1.e-8:
                    break
                k += 1
            if verbose:
                print(f"Energy at layer {j}, gate {i+j%2}: {E_gate[-1]}.")
                print(f"Fidelity at layer {j}, gate {i+j%2}: {F_gate[-1]}.")
                print(f"Pair fidelity at layer {j}, gate {i+j%2}: {FP_gate[-1]}.")
            E_layer.append(E_gate)
            F_layer.append(F_gate)
            FP_layer.append(FP_gate)
            mo_layer.append(mo_gate)
            if len(E_layer) > 1 and np.abs(np.mean(E_layer[-1]) - np.mean(E_layer[-2])) < 1.e-8:
                break
        Es.append(E_layer)
        Fs.append(F_layer)
        Fs_P.append(FP_layer)
        mos.append(mo_layer)
        if len(Es) > 2 and np.abs(np.mean(Es[-1][-1]) - np.mean(Es[-2][-1])) < 1.e-8:
            break
        psi = psi.transpose(np.roll(list(range(ell)), 1))

    return Es, Fs, Fs_P, mos, psi

In [21]:
L = 10
psi0 = np.random.rand(2**L) + 1.j * np.random.rand(2**L)
psi0 /= np.linalg.norm(psi0)

In [22]:
Es_C, Fs_C, Fs_P_C, mos_C, psi_C = HITE(psi0.reshape([2]*L), 10, h, 1, adaptive=True, force_fail=False, verbose=True, seed=-1, correct_iter=3, kill=0.)

Shifted Spectrum with shift 1.4142135623730943: [0.         0.41421356 2.41421356 2.82842712].
GS Energy: -12.784906442999315.
Energy at iteration 0: [[(7.415723523898057+4.440892098500626e-16j)]].
Fidelity at iteration 0: [[0.00010989005408390036]].
Pair fidelity at iteration 0: [[3.186819488254047e-05]].


  0%|          | 0/1 [00:00<?, ?it/s]

Measurement outcome and probability at round 0: 1, 0.8271843767730228.
Measurement outcome and probability at round 1: 0, 0.07317074101812956.
Measurement outcome and probability at round 2: 0, 0.9997531771046239.
Measurement outcome and probability at round 3: 0, 0.9999999376725478.
Measurement outcome and probability at round 4: 0, 0.9999999999842624.
Measurement outcome and probability at round 5: 0, 1.
Energy at layer 1, gate 1: (-2.133497585340458+0j).
Fidelity at layer 1, gate 1: 0.00010913093995640505.
Pair fidelity at layer 1, gate 1: 0.0017187670414162022.
Measurement outcome and probability at round 0: 1, 0.7479862391280999.
Measurement outcome and probability at round 0: 1, 0.6542815450759996.
Measurement outcome and probability at round 1: 0, 0.5716281880078665.
Measurement outcome and probability at round 2: 0, 0.9998108118373237.
Measurement outcome and probability at round 3: 0, 0.9999999522291619.
Measurement outcome and probability at round 4: 0, 0.9999999999879381.


100%|██████████| 1/1 [00:00<00:00,  7.38it/s]

Measurement outcome and probability at round 5: 0, 1.
Energy at layer 1, gate 3: (-4.222703061185525+1.1102230246251565e-16j).
Fidelity at layer 1, gate 3: 0.0017390236390034748.
Pair fidelity at layer 1, gate 3: 0.012849121913701635.
Measurement outcome and probability at round 0: 1, 0.8194907548611893.
Measurement outcome and probability at round 1: 0, 0.37449140703725475.
Measurement outcome and probability at round 2: 0, 0.9997771450991988.
Measurement outcome and probability at round 3: 0, 0.9999999437262695.
Measurement outcome and probability at round 4: 0, 0.9999999999857911.
Measurement outcome and probability at round 5: 0, 1.
Energy at layer 1, gate 5: (-6.604108484076782-5.551115123125783e-16j).
Fidelity at layer 1, gate 5: 0.03432381883560515.
Pair fidelity at layer 1, gate 5: 0.08298962973414248.
Measurement outcome and probability at round 0: 1, 0.7050219811458853.
Measurement outcome and probability at round 0: 1, 0.5938776722482955.
Measurement outcome and probability 

In [23]:
Fs_C[0]

[[0.00010989005408390036]]

In [24]:
[Fs_C[i][-1][-1] for i in range(2)]

[0.00010989005408390036, 0.46092785107804685]

In [25]:
Es_F, Fs_F, Fs_P_F, mos_F, psi_F = HITE(psi_C.reshape([2]*L), 0.01, h, 40, adaptive=True, force_fail=False, verbose=False, seed=-1, correct_iter=3)

In [26]:
[Fs_F[i][-1][-1] for i in range(41)]

[0.46092785107804696,
 0.5199058731566132,
 0.5174631424508787,
 0.5731025313619854,
 0.5687526146131783,
 0.6201676507945638,
 0.614488066162174,
 0.6612670808738375,
 0.654796524925994,
 0.6968763915636867,
 0.6900706859323733,
 0.7276194177963915,
 0.7208357413310842,
 0.7541579655883047,
 0.7476571557781364,
 0.7771252443874178,
 0.7710836317465982,
 0.7970913728551416,
 0.7916162571151756,
 0.8145499877840539,
 0.8096952476222824,
 0.8299174457099188,
 0.8256975032906713,
 0.8435387257512837,
 0.8399401815098976,
 0.8556962856943712,
 0.8526871536464966,
 0.8666196643577486,
 0.8641564384406311,
 0.8764946421566686,
 0.8745275379943049,
 0.885471402849045,
 0.8839481332592817,
 0.8936715069564749,
 0.8925399162796321,
 0.9011936865332721,
 0.9004035182080063,
 0.90811856794849,
 0.9076225870598742,
 0.9145124673026841,
 0.9142671121011962]

# Average Fidelity

In [32]:
realizations = 100
Fs = []
for i in tqdm(range(realizations)):
    L = 6
    psi0 = np.random.rand(2**L) + 1.j * np.random.rand(2**L)
    psi0 /= np.linalg.norm(psi0)

    Es_C, Fs_C, Fs_P_C, mos_C, psi_C = HITE(psi0.reshape([2]*L), 0.5, h, 10, adaptive=True, force_fail=False, verbose=False, seed=-1, correct_iter=2)
    Es_F, Fs_F, Fs_P_F, mos_F, psi_F = HITE(psi_C.reshape([2]*L), 0.01, h, 40, adaptive=True, force_fail=False, verbose=False, seed=-1, correct_iter=10)
    Fs.append([Fs_C[i][-1][-1] for i in range(11)] + [Fs_F[i][-1][-1] for i in range(1, 41)])

  7%|▋         | 7/100 [00:03<00:43,  2.13it/s]


KeyboardInterrupt: 

In [29]:
np.array(Fs).mean(axis=0)

array([0.00531663, 0.48372955, 0.51139892, 0.53560353, 0.51323824,
       0.52469358, 0.5304533 , 0.54277341, 0.58533251, 0.58085652,
       0.57466707, 0.49169188, 0.48459473, 0.50220049, 0.49807889,
       0.4913282 , 0.48252948, 0.46600205, 0.47351736, 0.42948231,
       0.41413027, 0.3932671 , 0.36148703, 0.33290447, 0.36592349,
       0.34083603, 0.33982806, 0.35348386, 0.38441304, 0.38078604,
       0.37798795, 0.38426731, 0.3979465 , 0.39514591, 0.40939987,
       0.40944891, 0.40717433, 0.39238007, 0.43059327, 0.43549698,
       0.44238484, 0.42347075, 0.41190311, 0.38930153, 0.36885224,
       0.36936463, 0.3492123 , 0.34523826, 0.36958685, 0.37870701,
       0.37065235])

In [30]:
np.array(Fs)[:,-1]

array([9.58569388e-01, 8.68510058e-01, 4.09116337e-01, 9.18402991e-01,
       9.91720214e-02, 4.56726802e-01, 9.86719472e-01, 5.86303256e-02,
       3.51168811e-01, 3.64285837e-01, 6.61044317e-02, 3.84360722e-01,
       7.82052157e-01, 9.95760362e-01, 7.04077777e-01, 1.26801930e-01,
       8.32901351e-01, 1.95106413e-03, 5.67694757e-01, 2.01519649e-01,
       1.30428180e-01, 2.68632335e-01, 4.90867603e-03, 6.20451645e-02,
       9.04287065e-03, 1.70066361e-01, 8.84850182e-01, 3.25425922e-02,
       5.08131935e-01, 1.48976801e-03, 9.69473894e-01, 6.97219440e-01,
       3.82011234e-01, 6.18164613e-02, 1.43767441e-01, 1.13465399e-01,
       9.32998692e-02, 1.32687180e-01, 7.80319815e-01, 9.75825691e-01,
       5.96086609e-02, 2.91804941e-01, 5.47992730e-02, 6.84146049e-02,
       7.11527201e-01, 3.99766220e-01, 9.51070643e-01, 1.39710449e-01,
       4.30648130e-01, 6.82977350e-02, 7.50325082e-02, 2.14452245e-01,
       6.51055564e-01, 8.42816371e-01, 6.76723291e-03, 1.54971235e-01,
      